<a href="https://colab.research.google.com/github/desaiankitb/pytorch-basics/blob/main/tutorial-pytorch-org/00_3_Building_the_model_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline

# Build a neural network

- Neural networks comprise of layers/modules that perform operations on data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to build your own neural netowrk. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily. 

- In the following sections, we will build a neural network to classify images in the FashionMNIST dataset. 

In [3]:
import os
import torch 
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get a hardware device for training 
- We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let's check to see if [torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we continue to use the CPU. 

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## Define the class
- We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the `forward` method. 

In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

- We create an instance of `NeuralNetwork`, and move it to the `device`, and print it's structure. 

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


- To use the model, we pass it the input data. This executes the model's `forward`, along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866). Do not call `model.forward()` directly! 

- Calling the model on the input returns a 10-dimentional tensor with raw predicted values for each class. We get the prediction densities by passing it through an instance of the `nn.Softmax` module. 

In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


## Model Layers
- Let's break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size **28x28** and see what happens to it as we pass it through the network. 

In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

- We initialize the [`nn.flatten`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).



In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

- The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) is a module that applies a linear transformation on the input using it's stored weights and biases. 

In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
- Non-linear activations are what create the complex mappings between the model's input and outputs. They are applied after linear transformations to introduce *nonlinearity*, helping neural networks learn a wide variety of phenomena. 

- In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our linear layers, but there are other activations to introduce non-linearity in your model. 

In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0025, -0.0661,  0.1621, -0.0492, -0.1363,  0.0035,  0.5103,  0.0418,
         -0.2492, -0.2430,  0.1269,  0.0299,  0.3455,  0.9130,  0.4395, -0.4852,
          0.1291, -0.4451,  0.1850,  0.3254],
        [ 0.0589,  0.0508,  0.1624, -0.2068, -0.3638, -0.3798,  0.5338,  0.0761,
         -0.2115, -0.2211,  0.0098,  0.1858,  0.4838,  0.3601,  0.3667, -0.5387,
          0.1836, -0.2116,  0.1000,  0.2927],
        [-0.1240, -0.1210,  0.1700, -0.1004, -0.2123, -0.0030,  0.8862,  0.0336,
         -0.3050,  0.0255,  0.0917,  0.2158,  0.2024,  0.4596,  0.5321, -0.5373,
         -0.1054, -0.2258,  0.3523,  0.1530]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.1621, 0.0000, 0.0000, 0.0035, 0.5103, 0.0418, 0.0000,
         0.0000, 0.1269, 0.0299, 0.3455, 0.9130, 0.4395, 0.0000, 0.1291, 0.0000,
         0.1850, 0.3254],
        [0.0589, 0.0508, 0.1624, 0.0000, 0.0000, 0.0000, 0.5338, 0.0761, 0.0000,
         0.0000, 0.0098, 0.1858, 0.4838, 0.3601, 0.366

### nn.Sequential
- [nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like `seq_modules`. 

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

###nn.Softmax
- The last linear layer of the neural network returns `logits` - raw values in [`-infty`, `infty`] - which are passed to the [nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values \[0, 1] representing the model's predicted densities for each class. `dim` parameter indicates the dimension along which the values must sum to 1. 

In [15]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)